# TODO
1. interaction
2. atrifical datasets
3. tests + compare with R package
4. find two datasets

In [8]:
import numpy as np
import pandas as pd
from numpy.linalg import inv
np.set_printoptions(suppress = True)

In [9]:
def logit(x):
    return np.log(x/(1-x)) 

def logit_der(x):
    return 1 / (x * (1 - x))

def logit_inv(x):
    return 1/(1+np.exp(-x)) 

In [183]:
data = '../../data/airquality.csv'

df = pd.read_csv(data)

X = df.drop(['Ozone'], axis=1)

y = df['Ozone'].reset_index(drop=True)



1. Get p = (p(x1,beta), p(x2,beta), ... , p(xn,beta))
2. Get W = diag((p(x1,beta)*(1-p(x1,beta)), p(x2,beta)*(1-p(x2,beta)), ... , p(xn,beta)*(1-p(x2,beta))))

In [233]:
class IWLS:

    def __init__(self):
        self.AIC = None
        self.BIC = None
        self.log_likelihood = []
        self.log_likelihood_path = []
        self.iterations = 0
        self.beta = None

    def fit(self, X, y, max_iter = 100, delta_limit = 0.01, standarize = True, interaction = None):

        was_array = isinstance(X, np.ndarray)
        if was_array:
            X = pd.DataFrame(X)
        colnames = X.columns.tolist()

        if isinstance(y, np.ndarray):
            y = pd.DataFrame({"y": y})

        if(standarize):
            X = (X - np.mean(X, axis=0)) / np.std(X,axis=0)

        if(self.beta is None):
            beta = X.iloc[0].copy(deep = True) * 0
            p = logit_inv(X @ beta)
            iteration = 0

        delta = 10000
        self.log_likelihood_path.append(self.Log_likelihood(p,y))
        while delta > delta_limit and iteration < max_iter:
            iteration += 1
            W = np.diag(p * (1-p))
            z = X @ beta + inv(W) @ (y - p)
            beta = inv(X.T @ W @ X) @ X.T @ W @ z
            beta = pd.DataFrame({"coeff" : beta.values}, index=colnames)
            p = logit_inv(np.array(X @ beta))

            self.log_likelihood_path.append(self.Log_likelihood(p,y))
            delta = np.abs(self.log_likelihood_path[iteration] - self.log_likelihood_path[iteration - 1])

        self.iterations = iteration
        self.log_likelihood = self.log_likelihood_path[iteration]

        if not was_array:
            self.beta = pd.DataFrame({"coeff" : beta}, index=colnames)
        else:
            self.beta = beta

    @staticmethod
    def Log_likelihood(p, y):
    
        # Log-likelihood of logistic regression
        L = y * np.log(p) + (1 - y) * np.log(1 - p)
        return sum(L)

In [234]:
iwls = IWLS()

In [235]:
X @ beta

,coeff
0,-0.162597
1,-0.070894
2,-1.235133
3,-1.511410
4,-0.528302
...,...
106,-3.761472
107,-0.726497
108,-2.506621
109,-0.702110


In [236]:
iwls.fit(X, y, standarize=False)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [180]:
iwls.log_likelihood

-45.23356238626713

In [181]:
iwls.beta

,coeff
0,0.056266
1,-0.238037
2,1.823974
3,-0.465634
4,0.481854


In [154]:
interactions = np.array([[1,2], [3,4]])

In [165]:
X.iloc[:,1] 

0     -2.868311
1     -2.190661
2      3.004655
3      1.762297
4     -1.513011
         ...   
106    7.522322
107   -3.433020
108    4.924664
109   -2.190661
110    1.762297
Name: 1, Length: 111, dtype: float64

In [54]:
for interaction in interactions:
    print(X.imag)

array([[1, 2],
       [3, 4]])

In [61]:
X = df.drop(['Ozone'], axis=1)

y = df['Ozone'].reset_index(drop=True)


In [57]:
X

,Solar.R,Wind,Temp,Month,Day
0,190,7.4,67,5,1
1,118,8.0,72,5,2
2,149,12.6,74,5,3
3,313,11.5,62,5,4
4,299,8.6,65,5,7
...,...,...,...,...,...
106,20,16.6,63,9,25
107,193,6.9,70,9,26
108,191,14.3,75,9,28
109,131,8.0,76,9,29


In [64]:
f(standarize):
            X = (X - np.mean(X, axis=0)) / np.std(X,axis=0)

        if(self.beta is None):
            beta = X.iloc[0].copy(deep = True) * 0
            p = logit_inv(X @ beta)
            iteration = 0

        delta = 10000
        self.log_likelihood_path.append(self.Log_likelihood(p,y))
        while delta > delta_limit and iteration < max_iter:
            iteration += 1
            W = np.diag(p * (1-p))
            z = X @ beta + inv(W) @ (y - p)
            beta = inv(X.T @ W @ X) @ X.T @ W @ z
            p = logit_inv(X @ beta)

            self.log_likelihood_path.append(self.Log_likelihood(p,y))
            delta = np.abs(self.log_likelihood_path[iteration] - self.log_likelihood_path[iteration - 1])

0      0
1      0
2      0
3      0
4      0
      ..
106    0
107    0
108    0
109    0
110    0
Name: Ozone, Length: 111, dtype: int64

['Solar.R', 'Wind', 'Temp', 'Month', 'Day']

In [114]:
beta.values

array([-0.00985555, -0.50057657,  1.11323779, -0.32261542,  0.23178149])

In [116]:
pd.DataFrame({"coeff" : beta.values}, index=X.columns.tolist())

,coeff
Solar.R,-0.009856
Wind,-0.500577
Temp,1.113238
Month,-0.322615
Day,0.231781


In [105]:
beta.rename(index = X.columns.tolist())

TypeError: Series.name must be a hashable type